In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
# !pip install --upgrade pip
# !pip install tensorflow 
# !pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas

    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      train_dataset = tf.data.Dataset.from_tensor_slices((input, output))
      train_dataset = train_dataset.shuffle(len(input))
      train_dataset = train_dataset.batch(self.batch_size).cache()
      train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (train_dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, random_state=42)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, random_state=42)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)


    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
)])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-23 23:02:14.047644: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 23:02:14.108681: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 23:02:14.108726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 23:02:14.110409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 23:02:14.119903: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 23:02:14.120877: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [41]:
from keras import saving
from keras import models

class Tester:
  def __init__(self, run_times=500):
    self.run_times = run_times
    self.better_metrics = {}

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    input_25, output_25 = split_df(df_25)
    input_1000, output_1000 = split_df(df_1000)
    trainer.split_dataset(input_25, output_25, input_1000, output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def Loop(self, sort_by, boundarie):
    betters = 0

    for i in range(self.run_times):
      print (f"++++++++++++++++++++++++++ {i + 1} ++++++++++++++++++++++++++")
      trainer = self.PrepareTraining()
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{betters}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        betters += 1

  def DisplayBetterResults(self, sort_by):
    df = pd.DataFrame.from_dict(self.better_metrics)
    df = df.transpose()
    df = df.sort_values([sort_by])
    display(df)
    excel_path = '../results/metrics.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [42]:
tester = Tester(run_times=250)
tester.Loop(sort_by='mse_sup', boundarie=2.5)

++++++++++++++++++++++++++ 1 ++++++++++++++++++++++++++


Elapsed time:  0.4794126340002549
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 2 ++++++++++++++++++++++++++
Elapsed time:  3.47706475799896
Stopped at epoch:  114
1/1 [==============================] - 0s 18ms/step
++++++++++++++++++++++++++ 3 ++++++++++++++++++++++++++
Elapsed time:  5.067080131999319
Stopped at epoch:  169
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 4 ++++++++++++++++++++++++++
Elapsed time:  0.7054612499996438
Stopped at epoch:  19
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 5 ++++++++++++++++++++++++++
Elapsed time:  2.1854890329996124
Stopped at epoch:  66
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 6 ++++++++++++++++++++++++++
Elapsed time:  3.0842997069994453
Stopped at epoch:  83
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 7 ++++++++++++++++++++++++++
Elapsed time:  0.5

2024-01-23 23:21:42.317372: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  27.38448153399986
Stopped at epoch:  971
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 98 ++++++++++++++++++++++++++
Elapsed time:  28.21027925100134
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 99 ++++++++++++++++++++++++++
Elapsed time:  3.111478453000018
Stopped at epoch:  105
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 100 ++++++++++++++++++++++++++
Elapsed time:  0.7939691790015786
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 101 ++++++++++++++++++++++++++
Elapsed time:  0.5978333260009094
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 102 ++++++++++++++++++++++++++
Elapsed time:  3.986518867999621
Stopped at epoch:  126
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 103 ++++++++++++++++++++++++++
Elapsed ti

In [43]:
tester.DisplayBetterResults('mse_sup')

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_6,0.843766,0.692675,0.044771,0.787592,0.675439,0.655682,2.055087,2.005239,1.972350,1.988794
model_12,0.862415,0.686918,-0.392577,0.867814,0.644960,0.577414,2.093583,2.383994,1.052549,1.718272
model_8,0.847191,0.669534,-0.884588,0.717629,0.631481,0.641308,2.209827,1.325264,2.390849,1.858057
model_4,0.843229,0.668077,0.649067,0.473423,0.606465,0.657936,2.219569,1.711946,2.310090,2.011018
model_10,0.815018,0.663830,-0.625813,0.678811,0.510723,0.776329,2.247973,2.394486,2.457570,2.426028
model_3,0.784311,0.658796,-0.430014,0.612368,0.388123,0.905200,2.281634,3.866386,1.661530,2.763958
model_0,0.837474,0.645031,0.236418,0.779971,0.729071,0.682087,2.373681,1.142018,3.121026,2.131523
model_1,0.804700,0.642994,0.496909,0.643088,0.640069,0.819632,2.387298,2.039450,2.397760,2.218605
model_9,0.815686,0.640028,0.253319,0.774363,0.600421,0.773527,2.407136,2.708684,1.440636,2.074660
model_2,0.795731,0.639961,-0.470490,0.701595,0.643691,0.857275,2.407579,1.099199,4.258771,2.678985


DataFrame salvo em ../results/metrics.xlsx
